In [1]:
"""
`COM.prettyprint(x)` prints nicely the object `x` 
for which `COM.prettify(x)` method is defined.
"""
module COM
prettify(x) = sprint(io -> show(io, "text/plain", x))
prettyprint(io::IO, x) = print(io, prettify(x))
prettyprint(x) = prettyprint(stdout, x)
end

using .COM
COM.prettyprint([π, 2π])

2-element Vector{Float64}:
 3.141592653589793
 6.283185307179586

In [2]:
"""A scirntific module (calculate exp(x))"""
module SCI

abstract type AbstractProblem end
struct Problem{T} <: AbstractProblem x::T end

abstract type AbstractAlgorithm end
struct Builtin <: AbstractAlgorithm end
Base.@kwdef struct Taylor <: AbstractAlgorithm n::Int = 10 end
default_algorithm(prob::Problem) = Builtin()

struct Solution{R, P<:AbstractProblem, A<:AbstractAlgorithm} result::R; prob::P; alg::A end
solve(prob::AbstractProblem) = solve(prob, default_algorithm(prob))
solve(prob::AbstractProblem, alg::Builtin) = Solution(exp(prob.x), prob, alg)
solve(prob::AbstractProblem, alg::Taylor) = Solution(sum(prob.x^k/factorial(k) for k in 0:alg.n), prob, alg)

using ..COM

COM.prettify(sol::Solution{R, P, A}) where {R, P<:AbstractProblem, A<:Builtin} = """
Problem:   x = $(sol.prob.x)
Algorithm: builtin exp(x)
Result:    $(sol.result)
"""

COM.prettify(sol::Solution{R, P, A}) where {R, P<:AbstractProblem, A<:Taylor} = """
Problem:   x = $(sol.prob.x)
Algorithm: Taylor series of exp(x) upto degree $(sol.alg.n)
Result:    $(sol.result)
"""

end

using .SCI
prob = SCI.Problem(1)
COM.prettyprint(SCI.solve(prob))
println()
COM.prettyprint(SCI.solve(prob, SCI.Taylor()))

Problem:   x = 1
Algorithm: builtin exp(x)
Result:    2.718281828459045

Problem:   x = 1
Algorithm: Taylor series of exp(x) upto degree 10
Result:    2.7182818011463845


In [3]:
"""Another scirntific module (calculate sin(x))"""
module SCI2

abstract type AbstractProblem end
struct Problem{T} <: AbstractProblem x::T end

abstract type AbstractAlgorithm end
struct Builtin <: AbstractAlgorithm end
Base.@kwdef struct Taylor <: AbstractAlgorithm n::Int = 1 end
default_algorithm(prob::Problem) = Builtin()

struct Solution{R, P<:AbstractProblem, A<:AbstractAlgorithm} result::R; prob::P; alg::A end
solve(prob::AbstractProblem) = solve(prob, default_algorithm(prob))
solve(prob::AbstractProblem, alg::Builtin) = Solution(sin(prob.x), prob, alg)
solve(prob::AbstractProblem, alg::Taylor) = Solution(sum((-1)^k*prob.x^(2k+1)/factorial(2k+1) for k in 0:alg.n), prob, alg)

using ..COM

COM.prettify(sol::Solution{R, P, A}) where {R, P<:AbstractProblem, A<:Builtin} = """
Problem:   x = $(sol.prob.x)
Algorithm: builtin sin(x)
Result:    $(sol.result)
"""

COM.prettify(sol::Solution{R, P, A}) where {R, P<:AbstractProblem, A<:Taylor} = """
Problem:   x = $(sol.prob.x)
Algorithm: Taylor series of sin(x) upto degree $(2sol.alg.n + 1)
Result:    $(sol.result)
"""

end

using .SCI2
prob2 = SCI2.Problem(π/6)
COM.prettyprint(SCI2.solve(prob2))
println()
COM.prettyprint(SCI2.solve(prob2, SCI2.Taylor()))

Problem:   x = 0.5235987755982988
Algorithm: builtin sin(x)
Result:    0.49999999999999994

Problem:   x = 0.5235987755982988
Algorithm: Taylor series of sin(x) upto degree 3
Result:    0.49967417939436376


[quote="BambOoxX, post:12, topic:64568"]
you define the `SCI` package such that it patches the `COM` package is this right ?
[/quote]

Yes.  It is a very standard pattern.

[quote="BambOoxX, post:12, topic:64568"]
If so : What would happen if `COM` is not loaded prior to `SC` .
[/quote]

Assume that, unlike the MWE I have shown above, both COM and SCI are packaged and available for `using COM` and `using SCI`.  Then the result of `using SCI; using COM` and the result of `using COM; using SCI` will be the same.

In the following example, `COM` package has a function `prettify(x)` that its methods can be defined for user-defined types, and a function `prettyprint(x)` that uses `prettify(x)`.  The `SCI` package makes use of them.